In [60]:
## Import Libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [61]:
## Using pandas, read html using the given url  
toronto_data=[]
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

toronto_df=pd.read_html(url, header=0)
toronto_df[0].to_csv('toronto.csv',index=False)
toronto_data=pd.read_csv('toronto.csv')

print (toronto_data.shape)
toronto_data.head()


(180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [62]:
## Remove rows with Borough == Not assigned

NABorough=toronto_data[toronto_data['Borough'] == 'Not assigned'].index
toronto_data.drop(NABorough, axis=0, inplace=True)
print(toronto_data.shape)
toronto_data.head()

(103, 3)


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [63]:
## install pgeocode to get latitudes and longitudes of postal codes
!pip install pgeocode
import pgeocode
pgeocode.Nominatim('ca')
geolocator = pgeocode.Nominatim('ca')
postal_codes = toronto_data['Postal Code'].tolist()
latitudes = []
longitudes = []
print("pgeocode downloaded")

pgeocode downloaded


In [64]:
for i, postal_code in enumerate(postal_codes):
    # initialize your variable to None
    #print(f'--Getting Postal Code: {postal_code}')
    g = geolocator.query_postal_code(postal_code)
    
    if not g.empty:
        #print(f'Postal Code {postal_code} has been retrieved. {len(postal_codes) - (i + 1)} codes left')
        latitudes.append(g.latitude)
        longitudes.append(g.longitude)


In [65]:
## check len of lists are the same before returning dataframe
print (len(postal_codes),len(latitudes),len(longitudes))

toronto_data['Neighbourhood Latitude']=latitudes
toronto_data['Neighbourhood Longitude']=longitudes
toronto_data.head()

103 103 103


,Postal Code,Borough,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude
2,M3A,North York,Parkwoods,43.7545,-79.3300
3,M4A,North York,Victoria Village,43.7276,-79.3148
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889


In [207]:
## Extract Rows of 2 Recommended Locations M4P and M4T using Boolean Function "=="
M4P = GTA['Postal Code']=='M4P'
M4T = GTA['Postal Code']=='M4T'
RecLoc=GTA[M4P|M4T]
RecLoc

,Postal Code,Borough,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude
67,M4P,Central Toronto,Davisville North,43.7135,-79.3887
82,M4T,Central Toronto,"Moore Park, Summerhill East",43.6899,-79.3853


In [208]:
## We made calls to Foursquare API to explore the 2 Recommended Locations for Promotion Partners
## We realized that the 2 Locations are adjacent, so we used M4P and simply expanded the LIMIT and radius  

CLIENT_ID = 'R12BNMHALD4Y5F1E5IWIA2YMYTTUZP4E0NZRUHVSMLA2WEIM' # your Foursquare ID
CLIENT_SECRET = 'UST5LXSGJNS0YH2ACP3JNUEIIUUIGOUONRC5VWZLZI5ZB3W0' # your Foursquare Secret
ACCESS_TOKEN = 'Q3RBZEJAR2E212U0MJS2NZTBBZ0ZGQB4Z4IKY2CMRSUWEYNX'
VERSION = '20180605' # Foursquare API version

## Define search query

LIMIT = 100 # A default Foursquare API limit value
address = 'M4P'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = 43.7135
longitude = -79.3887
radius = 1200

search_query = 'Gym'

url1 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

## Get results
results1 = requests.get(url1).json()
results1 

{'meta': {'code': 200, 'requestId': '604a5bb2c078f966fe41f801'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Davisville',
  'headerFullLocation': 'Davisville, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'query': 'gym',
  'totalResults': 17,
  'suggestedBounds': {'ne': {'lat': 43.724300010800015,
    'lng': -79.37378608022962},
   'sw': {'lat': 43.70269998919999, 'lng': -79.40361391977038}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '524ac4b1498e8498a174a9a0',
       'name': 'Crossfit Metric',
       'location': {'address': '756 Mt Pleasant Rd',
        'crossStreet': 'Eglinton',
  

In [209]:
L=(len(results1['response']['groups'][0]['items']))
print("There are",L,"venues that fit the Search Query for Promotion Partners in this Recommended Location")

There are 17 venues that fit the Search Query for Promotion Partners in this Recommended Location


In [210]:
## get relevant parts of JSON file returned and convert into a clean dataframe
# define function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

items = results1['response']['groups'][0]['items']
items[0]

dataframe = pd.json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns and drop those we do not need
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]
dataframe_filtered.drop(['cc','city','state','country'], axis=1, inplace=True)
print(dataframe_filtered.shape, dataframe_filtered.columns)

dataframe_filtered.head(17)

(17, 11) Index(['name', 'categories', 'address', 'crossStreet', 'lat', 'lng',
       'labeledLatLngs', 'distance', 'formattedAddress', 'postalCode', 'id'],
      dtype='object')


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,formattedAddress,postalCode,id
0,Crossfit Metric,Gym,756 Mt Pleasant Rd,Eglinton,43.707480,-79.389857,"[{'label': 'display', 'lat': 43.70747956557104...",676,"[756 Mt Pleasant Rd (Eglinton), Toronto ON, Ca...",NaN,524ac4b1498e8498a174a9a0
1,GoodLife Fitness Toronto Dunfield and Eglinton,Gym,110 Eglinton Ave E,at Dunfield Ave.,43.707645,-79.395303,"[{'label': 'display', 'lat': 43.707645, 'lng':...",840,"[110 Eglinton Ave E (at Dunfield Ave.), Toront...",M4P 1A6,4bd6e35d637ba5933ad3f870
2,Barreworks,Yoga Studio,2576 Yonge St,NaN,43.714070,-79.400109,"[{'label': 'display', 'lat': 43.71407030751952...",920,"[2576 Yonge St, Toronto ON, Canada]",NaN,51606062e4b0878cf540f4a2
3,GoodLife Fitness Toronto Yonge Eglinton Centre,Gym,2300 Yonge St,at Eglinton Ave. W,43.707276,-79.399562,"[{'label': 'display', 'lat': 43.707276, 'lng':...",1115,"[2300 Yonge St (at Eglinton Ave. W), Toronto O...",M4P 1E4,4e80de0930f8844fb2bd7503
4,Gym,Gym,140 Erskine,NaN,43.713126,-79.393537,"[{'label': 'display', 'lat': 43.71312601210131...",391,"[140 Erskine, Toronto ON, Canada]",NaN,4da99b34a86e771ea70e84c1
5,900 Mount Pleasant - Residents Gym,Gym / Fitness Center,900 Mount Pleasant Road,NaN,43.711671,-79.391767,"[{'label': 'display', 'lat': 43.71167058860572...",319,"[900 Mount Pleasant Road, Toronto ON M4P 3J9, ...",M4P 3J9,4c3f2724db3b1b8d635e6695
6,Yoga Tree Midtown,Yoga Studio,40 Eglinton Ave. E,at Yonge St.,43.707642,-79.397472,"[{'label': 'display', 'lat': 43.70764167668336...",960,"[40 Eglinton Ave. E (at Yonge St.), Toronto ON...",M4P 3A2,4efa4c786c25c411edcaf286
7,Womens Fitness Clubs of Canada,Gym,1820 Bayview Ave Unit 1,NaN,43.712484,-79.377341,"[{'label': 'display', 'lat': 43.71248383171391...",920,"[1820 Bayview Ave Unit 1, Toronto ON M4G 4G7, ...",M4G 4G7,4bc6fb8b92b376b091d54e3a
8,Yoga Plus,Gym,40 Eglinton Ave E,at Yonge,43.707674,-79.397478,"[{'label': 'display', 'lat': 43.707674, 'lng':...",958,"[40 Eglinton Ave E (at Yonge), Toronto ON, Can...",NaN,4b156d91f964a520b9ac23e3
9,CYCLEBAR,Gym,"1866 Bayview Avenue, Suite 103",NaN,43.714468,-79.378066,"[{'label': 'display', 'lat': 43.71446846114807...",862,"[1866 Bayview Avenue, Suite 103, Toronto ON M4...",M4G 0C3,58cd67d92f91cb28ec3d684e


In [214]:
## Generate a map of all Promotion Partners in the Recommended Location  

venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) 
# add venues from Search Query as blue circle markers
for lat, lng, label, name in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories, dataframe_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label+"--"+name,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map  


In [259]:
## Explore Promotion Partners for Users and Tips. 
## We cycled through each venue ID and chose the one with Highest Rating
venue_id = '4efa4c786c25c411edcaf286' # ID of YogaPlus
urlr = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
result = requests.get(urlr).json()  
print(result['response']['venue'].keys())
result['response']['venue'] 


dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4efa4c786c25c411edcaf286',
 'name': 'Yoga Tree Midtown',
 'contact': {'phone': '4163229642',
  'formattedPhone': '(416) 322-9642',
  'twitter': 'yoga_tree'},
 'location': {'address': '40 Eglinton Ave. E',
  'crossStreet': 'at Yonge St.',
  'lat': 43.70764167668336,
  'lng': -79.3974716381371,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.70764167668336,
    'lng': -79.3974716381371}],
  'postalCode': 'M4P 3A2',
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['40 Eglinton Ave. E (at Yonge St.)',
   'Toronto ON M4P 3A2',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/yoga-tree-midtown/4efa4c786c25c411edcaf286',
 'categories': [{'id': '4bf58dd8d48988d102941735',
   'name': 'Yoga Studio',
   'pluralName': 'Yoga Studios',
   'shortName': 'Yoga Studio',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gym_yogastudio_',
    'suffix': '.png'},
   'primary': True}],
 'verified': True,
 'stats': {'t

In [290]:
## We found the highest rating
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet')
          

8.1


In [293]:
## we used "tips?" to get a JSON file with Tips for venue
venue_id = '4efa4c786c25c411edcaf286' # ID of YogaTree Midtown  
limit = 20 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION, limit)
resulttips = requests.get(url).json()
resulttips 
tips = resulttips['response']['tips']['items']
tip = resulttips['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'likes', 'like', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user'])

In [294]:
pd.set_option('display.max_colwidth', None)

tips_df = pd.json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered
 
  

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.id
0,Try the 30 day unlimited classes for $40...great new studio.,1,0,4f302be9e4b07ca31811dc13,Judes,C,960405


In [288]:
## We searched User ID and identified a KOL with frequent tips, high agreeCount and zero disagreeCount

idnumber = '960405' # user ID of Judes C  

url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}'.format(idnumber, CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN,VERSION) 

# send GET request
resultusers = requests.get(url).json()
#user_data = results['response']['user']
tips=resultusers['response']['tips']['items']
## [0]['venue']['photos']['groups'][0]['items']
# display features associated with users

# format column width
pd.set_option('display.max_colwidth', None)
tips_df = pd.json_normalize(tips)
# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]
# display user's tips
tips_filtered
 


,text,agreeCount,disagreeCount,id
0,"Beef Bolognaise, white chocolate score cheesecake = SUPREME. Try the grapefruit lager. Best this summer #foodie #saucy",1,0,5399350211d2b5f1381bdd3f
1,So much fun! Gotta come back with more friends #dodgeball #foampitshouldhavemorelanes,0,0,530041ad11d24ed4822a569e
2,Came out of the parking garage & found a better parking area that's a lot closer to the path. #morningglory,1,0,4f915933e4b048b2e456c9cd
3,Try the 30 day unlimited classes for $40...great new studio.,1,0,4f302be9e4b07ca31811dc13
4,Get here early to avoid the long lines.,1,0,4f1eb59ce4b0d881fd7d6a10
5,Everything I need in one place...good place to get all your vegetables,0,0,4f0fa53fe4b06c5888c1b0f6
6,Must try the Truffle Brie and the 12yr old chedder...WOW!,0,0,4ef3cbd649010be35db4d44d
7,Technique is king! Great instructors.,0,0,4ee03ffcb8f741383026f77d
8,"Good whiskey, good vodka and good vibes are welcome here.",0,0,4eb71bd5f5b94bd85c88fbd4
9,"it's not just a donut shop, it's Canada's donut shop",1,0,4e71ea07aeb79ea0952497bd


In [306]:
print("Client V: we recommend contacting Gyms/Yoga Studios on the Promotion Partners List for joint marketing.\n")
print("They are all within 1000m radius of your Location, affording good source of foot traffic within 10 mins walk.\n")
print("We also recommend inviting Judes C who is a KOL with good following and many AgreeCounts for food tastings.\n") 
print("++++++++++ GOOD LUCK ++++++++++++")

Client V: we recommend contacting Gyms/Yoga Studios on the Promotion Partners List for joint marketing.

They are all within 1000m radius of your Location, affording good source of foot traffic within 10 mins walk.

We also recommend inviting Judes C who is a KOL with good following and many AgreeCounts for food tastings.

++++++++++ GOOD LUCK ++++++++++++
